In [2]:
from __future__ import print_function, division
from IPython.display import clear_output
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation,Conv2DTranspose
from keras.layers.convolutional import Conv2D
from keras.models import Sequential, Model
from functools import partial
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import dnslib
import json
import tensorflow as tf
import os
from tensorflow.keras.layers import LeakyReLU
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [3]:
def encode(dataset):
    d = 2
    w = 16
    encodedData = []

    for packet in dataset:
        converted = []

        for num in packet:
            converted.append(int((int(num, base=16) * w) + (w / 2)))

        matrix = np.zeros((28, 28))
        r, c = 0, 0

        for digit in converted:
            matrix[r][c] = digit
            matrix[r][c + 1] = digit
            matrix[r + 1][c] = digit
            matrix[r + 1][c + 1] = digit

            if (c + 1) % 27 == 0:
                c = 0
                r += d
            else:
                c += d
            encodedData.append(matrix)
    return encodedData

In [4]:
def decode(encodedMatrix):

    d = 2
    w = 16
    decodedData = []
    r, c = 0, 0


    avgArray = []

    for _ in range(int((28 * 28) / 4)):
        avgArray.append((encodedMatrix[r][c] + encodedMatrix[r][c + 1] + encodedMatrix[r + 1][c] +
                         encodedMatrix[r + 1][c + 1]) / 4)

        if (c + 1) % 27 == 0:
            c = 0
            r += d
        else:
            c += d

    decodedArray = []

    for val in avgArray:
        byte = hex(int(val))
        high = int(byte[2:3], 16)

        if high <= 15:
            low = 0
        else:
            low = int(byte[3:4], 16)

        decodedArray.append(hex(int(((high / w) * w) + (low / w)))[2:])

    decodedData.append(decodedArray)

    return decodedArray

In [4]:
dataset = []

#raw packet capture file
'''
with open("dns_samples_5480.json") as f:
    frames_json = json.load(f)
    # extract hex streams
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 1 loaded")
'''
with open("dns_samples_6593.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 2 loaded")

with open("dns_samples_7224.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 3 loaded")

with open("dns_samples_7486.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 4 loaded")

#split up dataset encoding so dont run out of memory
X_train1 = np.array(encode(dataset))
# Rescale -1 to 1
X_train1 = (X_train1.astype(np.float32) - 127.5) / 127.5
dataset=[]

with open("ping_800.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 5 loaded")

with open("dns_samples_7900.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("Dataset 6 loaded")

with open("dns_samples_13068.json") as f:
    frames_json = json.load(f)
    hexStreams = [frame["_source"]["layers"]["frame_raw"][0] for frame in frames_json]

    for stream in hexStreams:
        if len(stream)<=196:
            hexArray = [stream[i:i + 1] for i in range(28, len(stream), 1)]
            dataset.append(hexArray)
print("All datasets loaded")

X_train = np.array(encode(dataset))
print("Dataset encoded")
# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.concatenate((X_train1, X_train))
X_train = np.expand_dims(X_train, axis=3)
dataset=[]
X_train1=[]

Dataset 1 loaded
Dataset 2 loaded
Dataset 3 loaded
Dataset 4 loaded
Dataset 5 loaded
Dataset 6 loaded
All datasets loaded
Dataset encoded


In [5]:
def printPacket(packet, epoch):
    print("IP Version: ", packet[0])
    print("Header Length: ", packet[1])
    print("Differentiated Services Field: ", hex(int((str(packet[2])+str(packet[3])), 16)))
    print("Total Length: ", int((str(packet[4])+str(packet[5])+str(packet[6])+str(packet[7])), 16))
    print("Identification: ", hex(int((str(packet[8])+str(packet[9])+str(packet[10])+str(packet[11])), 16)))
    print("Flags: ", hex(int((str(packet[12])+str(packet[13])), 16)))
    print("Fragment Offset: ", hex(int((str(packet[14])+str(packet[15])), 16)))
    print("Time to live: ", int((str(packet[16])+str(packet[17])), 16))
    print("Protocol: ", int((str(packet[18])+str(packet[19])), 16))
    print("Header checksum: ", hex(int((str(packet[20])+str(packet[21])+str(packet[22])+str(packet[23])), 16)))
    print("Source: ", (str(int((str(packet[24])+str(packet[25])), 16)))+"."+str(int((str(packet[26])+str(packet[27])), 16))+"."+str(int((str(packet[28])+str(packet[29])), 16))+"."+str(int((str(packet[30])+str(packet[31])), 16)))
    print("Destination: ",(str(int((str(packet[32])+str(packet[33])), 16)))+"."+str(int((str(packet[34])+str(packet[35])), 16))+"."+str(int((str(packet[36])+str(packet[37])), 16))+"."+str(int((str(packet[38])+str(packet[39])), 16)))
    print("----------------------------------")
    print("Source Port: ", int((str(packet[40])+str(packet[41])+str(packet[42])+str(packet[43])), 16))
    print("Destination Port: ", int((str(packet[44])+str(packet[45])+str(packet[46])+str(packet[47])), 16))
    print("Length: ", int((str(packet[48])+str(packet[49])+str(packet[50])+str(packet[51])), 16))
    print("Checksum: ", hex(int((str(packet[52])+str(packet[53])+str(packet[54])+str(packet[55])), 16)))
    print("----------------------------------")

    try:
        packet[0]=4
        packet[14]=0
        packet[15]=0
        #make checksum 0
        packet[52]=0
        packet[53]=0
        packet[54]=0
        packet[55]=0
        dns_string=[]
        sep=''
        for i in range(56, len(packet)):
            dns_string.append(packet[i])
        dns_string = sep.join(dns_string)
        p = dnslib.binascii.unhexlify(dns_string)
        d = dnslib.DNSRecord.parse(p)
        print(d)

        wgan.generator.save("GoodModels/generator-gp%d" % epoch)
        wgan.critic.save("GoodModels/critic-gp%d" % epoch)
    except:
        print("DNS Malformed")
        print("----------------------------------")
        print("Transaction ID ", hex(int((str(packet[56])+str(packet[57])+str(packet[58])+str(packet[59])), 16)))
        print("Flags: ", hex(int((str(packet[60])+str(packet[61])+str(packet[62])+str(packet[63])), 16)))
        print("Questions: ", int((str(packet[64])+str(packet[65])+str(packet[66])+str(packet[67])), 16))
        print("Answer RR's: ", int((str(packet[68])+str(packet[69])+str(packet[70])+str(packet[71])), 16))
        print("Authority RR's: ", int((str(packet[72])+str(packet[73])+str(packet[74])+str(packet[75])), 16))
        print("Additional RR's: ", int((str(packet[76])+str(packet[77])+str(packet[78])+str(packet[79])), 16))

        #print("Hostname: ", bytes.fromhex(dns_string).decode('utf-8'))
        #print("Type: ", int((str(packet[len(packet)-8])+str(packet[len(packet)-7])+str(packet[len(packet)-6])+str(packet[len(packet)-5])), 16))
        #print("Class: ", int((str(packet[len(packet)-4])+str(packet[len(packet)-3])+str(packet[len(packet)-2])+str(packet[len(packet)-1])), 16))

        dns_string=[]
        sep=''
        for i in range(80, len(packet)):
            dns_string.append(packet[i])
        dns_string = sep.join(dns_string)
        print(dns_string)


In [6]:
class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 64

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.5)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        noise = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(noise)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        BATCH_SIZE=32
        alpha = K.random_uniform(shape = [BATCH_SIZE, 28, 28, 1], minval=0, maxval=1.0)
        interpolated_img = real_img * alpha + (fake_img * (1-alpha))

        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)

        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, noise],
                                outputs=[valid, fake, validity_interpolated])

        self.critic_model.compile(loss=[self.wasserstein_loss, self.wasserstein_loss, partial_gp_loss],
                                            optimizer=optimizer,
                                            loss_weights=[1, 1, 10])

        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = 10 * K.square(gradient_l2_norm-1)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(64, use_bias=False, input_shape=(64,), kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        #model.add(BatchNormalization())
        model.add(LeakyReLU())

        model.add(Dense(1024, use_bias=False, input_shape=(64,), kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())

        model.add(Dense(12544, use_bias=False, input_shape=(1024,), kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())

        model.add(Reshape((7, 7, 256)))

        model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())

        model.add(Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False, kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        #model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())

        model.add(Conv2D(1, (4, 4), strides=1, padding='same', use_bias=False))
        model.add(Activation('tanh'))
        model.summary()

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=4, strides=2, input_shape=self.img_shape, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        model.add(LeakyReLU())
        model.add(Dropout(0.25))

        model.add(Conv2D(32, kernel_size=4, strides=2, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        model.add(LeakyReLU())
        model.add(Dropout(0.25))

        model.add(Conv2D(64, kernel_size=4, strides=2, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        model.add(LeakyReLU())
        model.add(Dropout(0.25))

        model.add(Conv2D(128, kernel_size=4, strides=2, padding="same", kernel_regularizer=tf.keras.regularizers.l2(0.000025)))
        model.add(LeakyReLU())
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # real img class labels
        valid = -np.ones((batch_size, 1))
        #fake image class labels
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        loss =[]
        loss1 = []
        for epoch in range(epochs):

            for _ in range(self.n_critic):

                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                imgs = X_train[idx]
                # Sample generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                # Train the critic
                d_loss = self.critic_model.train_on_batch([imgs, noise], [valid, fake, dummy])

            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.generator_model.train_on_batch(noise, valid)
            loss.append(d_loss[0])
            loss1.append(d_loss[1])

            if epoch%25==0:
                # Plot the progress
                print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

            if epoch%100==0:
                self.plotLoss(loss, loss1)

    def sample_images(self, epoch):
            noise = np.random.normal(0, 1, (1, 64))
            gen_pkt = self.generator.predict(noise)
            gen_pkt = 127.5*gen_pkt[0,:,:,0]+127.5
            gen_pkt=decode(np.array(gen_pkt).astype("int"))

            print("Epoch: ", epoch, "-------------------------------------------------")
            printPacket(gen_pkt, epoch)

    def plotLoss(self, loss, loss1):
        #clear_output(wait=True)
        fig, ax = plt.subplots(dpi = 120)
        ax.set_ylabel("Critic Loss")
        ax.set_xlabel("Epoch")
        ax.plot(np.linspace(0, len(loss), len(loss)),loss, 'r')
        ax.plot(np.linspace(0, len(loss1), len(loss1)),loss1, 'b')
        plt.ylim([-100, 200])
        plt.show()



In [ ]:
print("Training Started")
wgan = WGANGP()
wgan.train(epochs=19000, batch_size=32, sample_interval=100)

In [7]:
genModel = keras.models.load_model("GoodModels/generator-gp17800", compile=False)

In [ ]:

def genPkt(packet):

        try:
            packet[14]=0
            packet[15]=0
            #make checksum 0
            packet[52]=0
            packet[53]=0
            packet[54]=0
            packet[55]=0
            #packet[0]=4
            dns_string=[]
            sep=''
            for i in range(56, len(packet)):
                dns_string.append(packet[i])
            dns_string = sep.join(dns_string)
            p = dnslib.binascii.unhexlify(dns_string)
            d = dnslib.DNSRecord.parse(p)

            print("--------------IP Header------------")
            print("IP Version: ", packet[0])
            print("Header Length: ", packet[1])
            print("Differentiated Services Field: ", hex(int((str(packet[2])+str(packet[3])), 16)))
            print("Total Length: ", int((str(packet[4])+str(packet[5])+str(packet[6])+str(packet[7])), 16))
            print("Identification: ", hex(int((str(packet[8])+str(packet[9])+str(packet[10])+str(packet[11])), 16)))
            print("Flags: ", hex(int((str(packet[12])+str(packet[13])), 16)))
            print("Fragment Offset: ", hex(int((str(packet[14])+str(packet[15])), 16)))
            print("Time to live: ", int((str(packet[16])+str(packet[17])), 16))
            print("Protocol: ", int((str(packet[18])+str(packet[19])), 16))
            print("Header checksum: ", hex(int((str(packet[20])+str(packet[21])+str(packet[22])+str(packet[23])), 16)))
            print("Source: ", (str(int((str(packet[24])+str(packet[25])), 16)))+"."+str(int((str(packet[26])+str(packet[27])), 16))+"."+str(int((str(packet[28])+str(packet[29])), 16))+"."+str(int((str(packet[30])+str(packet[31])), 16)))
            print("Destination: ",(str(int((str(packet[32])+str(packet[33])), 16)))+"."+str(int((str(packet[34])+str(packet[35])), 16))+"."+str(int((str(packet[36])+str(packet[37])), 16))+"."+str(int((str(packet[38])+str(packet[39])), 16)))
            print("---------------UDP Header-------------")
            print("Source Port: ", int((str(packet[40])+str(packet[41])+str(packet[42])+str(packet[43])), 16))
            print("Destination Port: ", int((str(packet[44])+str(packet[45])+str(packet[46])+str(packet[47])), 16))
            print("Length: ", int((str(packet[48])+str(packet[49])+str(packet[50])+str(packet[51])), 16))
            print("Checksum: ", hex(int((str(packet[52])+str(packet[53])+str(packet[54])+str(packet[55])), 16)))
            print("--------------DNS Packet--------------")

            print(d)
            return 1
        except:
            return 0

count=0
for _ in range(100):
    #noise = np.random.normal(0, 1, (1, 64))
    noise = np.random.uniform(0, 1, (1, 64))
    gen_pkt = genModel.predict(noise)
    gen_pkt = ((127.5*gen_pkt[0,:,:,0])+127.5).astype('int')
    gen_pkt=decode(np.array(gen_pkt).astype("int"))
    count+=genPkt(gen_pkt)

print("Success Rate: ", count/100)


c:\users\spencer\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
